### Connexion de Google Drive

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow
%cd /content/drive/MyDrive/journaux-xix-siecle/main"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import

In [ ]:
import cv2
import numpy as np
from scipy import signal
import os
from os import listdir

### Fonctions

In [ ]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def contour(image):
    kernel_LR = np.array([[1, 0, -1],
                          [2, 0, -2],
                          [1, 0, -1]])
    img1 = signal.convolve2d(image, kernel_LR, boundary='symm', mode='same')

    kernel_UD = np.array([[1, 2, 1],
                          [0, 0, 0],
                          [-1, -2, -1]])
    img2 = signal.convolve2d(image, kernel_UD, boundary='symm', mode='same')

    return np.absolute(img1) + np.absolute(img2)
    

def triangle(image):
  image_copy = image.copy()

  for x in range(2, image.shape[0]-2):
    for y in range(2, image.shape[1]-2):

      if image[x, y] == 255 and image[x, y+1] == 255 and image[x+1, y] == 255 and image[x+1, y+1] == 255:
        image_copy[x-1, y] = 255
        image_copy[x-1, y+1] = 255
        image_copy[x, y-1] = 255
        image_copy[x, y+2] = 255
        image_copy[x+1, y-1] = 255
        image_copy[x+1, y+2] = 255
        image_copy[x+2, y] = 255
        image_copy[x+2, y+1] = 255
        
  return image_copy


def remplissage(image):
  image_copy = image.copy()

  for x in range(2, image.shape[0]-2):
    for y in range(2, image.shape[1]-2):

      if image[x-1, y] == 0 and image[x, y+1] == 0 and image[x+1, y] == 0 and image[x, y-1] == 0:
        image_copy[x, y] = 0
        
  return image_copy

"def getImageExtension(imageString):\n  if imageString.endswith('.jpg'):\n    return '.jpg'\n  if imageString.endswith('.jpeg'):\n    return '.jpeg'\n  if imageString.endswith('.tif'):\n    return '.tif'\n  if imageString.endswith('.jpg'):\n    return '.jpg'"

### Traitement d'image


In [ ]:
dir = 'images'
images = []

for image in os.listdir(dir):
  if image.endswith(".jpg") or image.endswith(".tif") or image.endswith(".jpeg") or image.endswith(".png"):
    images.append(image)

In [ ]:
for image in images:

  img = cv2.imread(dir + '/' + image)
  gray = get_grayscale(img)
  
  bilateral = cv2.bilateralFilter(gray, 15, 50, 120)
  edges = contour(bilateral)
  _, thresh = cv2.threshold(gray, 10, 255, 1)
  img = (edges + 1.5*thresh)

  # Image négative
  data = np.full(img.shape, 255) 
  data[img > 180] = 0
  img = data


  img = triangle(img)
  img = remplissage(img)

  #cv2_imshow(img)

  imgStringSplit = image.split('.')

  cv2.imwrite(dir + '/' + imgStringSplit[0] + '_traitee' + '.' + imgStringSplit[1], img)
  os.remove(dir + '/' + image)
